In [1]:
import numpy as np 
import pandas as pd 
import datetime as dt 

In [2]:
train_df = pd.read_csv('../../../data/data/train.csv')

In [5]:
train_df.info()
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27499 entries, 0 to 27498
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Reservation-id          27499 non-null  int64 
 1   Gender                  27499 non-null  object
 2   Age                     27499 non-null  int64 
 3   Ethnicity               27499 non-null  object
 4   Educational_Level       27499 non-null  object
 5   Income                  27499 non-null  object
 6   Country_region          27499 non-null  object
 7   Hotel_Type              27499 non-null  object
 8   Expected_checkin        27499 non-null  object
 9   Expected_checkout       27499 non-null  object
 10  Booking_date            27499 non-null  object
 11  Adults                  27499 non-null  int64 
 12  Children                27499 non-null  int64 
 13  Babies                  27499 non-null  int64 
 14  Meal_Type               27499 non-null  object
 15  Vi

,Reservation-id,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Expected_checkin,Expected_checkout,...,Meal_Type,Visted_Previously,Previous_Cancellations,Deposit_type,Booking_channel,Required_Car_Parking,Reservation_Status,Use_Promotion,Discount_Rate,Room_Rate
0,39428300,F,40,Latino,Grad,<25K,North,City Hotel,7/1/2015,7/2/2015,...,BB,No,No,No Deposit,Online,Yes,Check-In,Yes,10,218
1,77491756,F,49,Latino,Mid-School,50K -- 100K,East,City Hotel,7/1/2015,7/2/2015,...,BB,No,No,Refundable,Online,Yes,Check-In,No,0,185
2,73747291,F,42,caucasian,Grad,<25K,East,City Hotel,7/2/2015,7/6/2015,...,BB,No,No,No Deposit,Online,Yes,Check-In,No,0,119
3,67301739,M,25,African American,College,>100K,South,Airport Hotels,7/2/2015,7/3/2015,...,BB,No,No,Refundable,Agent,Yes,Check-In,Yes,5,144
4,77222321,F,62,Latino,High-School,25K --50K,East,Resort,7/3/2015,7/4/2015,...,BB,No,No,No Deposit,Direct,No,Check-In,Yes,10,242


In [3]:
train_df.columns

Index(['Reservation-id', 'Gender', 'Age', 'Ethnicity', 'Educational_Level',
       'Income', 'Country_region', 'Hotel_Type', 'Expected_checkin',
       'Expected_checkout', 'Booking_date', 'Adults', 'Children', 'Babies',
       'Meal_Type', 'Visted_Previously', 'Previous_Cancellations',
       'Deposit_type', 'Booking_channel', 'Required_Car_Parking',
       'Reservation_Status', 'Use_Promotion', 'Discount_Rate', 'Room_Rate'],
      dtype='object')

In [4]:
def date_type_conversion( df ,columns):

    for each in columns:
        for index in df.index:
                val_in = str(df.loc[index,each]).strip().split('/')
                df.loc[index,each] = '/'.join([val_in[1],val_in[0],val_in[2]])
            #print('/'.join([check_in[1],check_in[0],check_in[2]]))

        df[each] = pd.to_datetime(df[each],dayfirst=True)
    return df

In [5]:
def split_date(df,columns):
    df = df.copy()
    for each in columns:
        df['day_'+each] = df[each].map(lambda x:x.day)
        df['month_'+each] = df[each].map(lambda x:x.month)
        df['year_'+each] = df[each].map(lambda x:x.year)
        df['dayofweek_'+each] = df[each].map(lambda x:x.dayofweek)
    return df

In [6]:
cnv_tr_df = date_type_conversion(train_df, ['Expected_checkin',
       'Expected_checkout', 'Booking_date'])
new_df = split_date(cnv_tr_df, ['Expected_checkin'])

In [8]:
new_df.head()

,Reservation-id,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Expected_checkin,Expected_checkout,...,Booking_channel,Required_Car_Parking,Reservation_Status,Use_Promotion,Discount_Rate,Room_Rate,day_Expected_checkin,month_Expected_checkin,year_Expected_checkin,dayofweek_Expected_checkin
0,39428300,F,40,Latino,Grad,<25K,North,City Hotel,2015-07-01,2015-07-02,...,Online,Yes,Check-In,Yes,10,218,1,7,2015,2
1,77491756,F,49,Latino,Mid-School,50K -- 100K,East,City Hotel,2015-07-01,2015-07-02,...,Online,Yes,Check-In,No,0,185,1,7,2015,2
2,73747291,F,42,caucasian,Grad,<25K,East,City Hotel,2015-07-02,2015-07-06,...,Online,Yes,Check-In,No,0,119,2,7,2015,3
3,67301739,M,25,African American,College,>100K,South,Airport Hotels,2015-07-02,2015-07-03,...,Agent,Yes,Check-In,Yes,5,144,2,7,2015,3
4,77222321,F,62,Latino,High-School,25K --50K,East,Resort,2015-07-03,2015-07-04,...,Direct,No,Check-In,Yes,10,242,3,7,2015,4


In [22]:
def stay_period(df,
                checkin_column='Expected_checkin',
                checkout_column = 'Expected_checkout',
                stay_column = 'stay'):

     ser_num = df[checkout_column] - df[checkin_column]
     nums = ser_num.dt.days.values
     df[stay_column] = pd.Series(data=nums, index=ser_num.index)
     return df

def booking_to_checkin(df,
                       booking_column= 'Booking_date',
                       checkin_column='Expected_checkin',
                       duration_column = 'booking_to_checkin'
                       ):
    ser_num = df[checkin_column]-df[booking_column]
    nums = ser_num.dt.days.values
    df[duration_column] = pd.Series(data=nums, index=ser_num.index)
    return df

In [23]:
nw1_df = stay_period(new_df)

In [24]:
nw2_df = booking_to_checkin(nw1_df)

In [25]:
nw2_df.head()

,Reservation-id,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Expected_checkin,Expected_checkout,...,Reservation_Status,Use_Promotion,Discount_Rate,Room_Rate,day_Expected_checkin,month_Expected_checkin,year_Expected_checkin,dayofweek_Expected_checkin,stay,booking_to_checkin
0,39428300,F,40,Latino,Grad,<25K,North,City Hotel,2015-07-01,2015-07-02,...,Check-In,Yes,10,218,1,7,2015,2,1,41
1,77491756,F,49,Latino,Mid-School,50K -- 100K,East,City Hotel,2015-07-01,2015-07-02,...,Check-In,No,0,185,1,7,2015,2,1,36
2,73747291,F,42,caucasian,Grad,<25K,East,City Hotel,2015-07-02,2015-07-06,...,Check-In,No,0,119,2,7,2015,3,4,3
3,67301739,M,25,African American,College,>100K,South,Airport Hotels,2015-07-02,2015-07-03,...,Check-In,Yes,5,144,2,7,2015,3,1,12
4,77222321,F,62,Latino,High-School,25K --50K,East,Resort,2015-07-03,2015-07-04,...,Check-In,Yes,10,242,3,7,2015,4,1,13


In [26]:
def total_num_participants(df,columns = ['Adults','Children','Babies'],column_name='total_participants'):
    df[column_name] = df[columns[0]]
    for each in columns[1:]:
        df[column_name] =df[column_name]+df[each]

    return df

In [27]:
nw3_df = total_num_participants(nw2_df)

In [28]:
nw3_df.head()

,Reservation-id,Gender,Age,Ethnicity,Educational_Level,Income,Country_region,Hotel_Type,Expected_checkin,Expected_checkout,...,Use_Promotion,Discount_Rate,Room_Rate,day_Expected_checkin,month_Expected_checkin,year_Expected_checkin,dayofweek_Expected_checkin,stay,booking_to_checkin,total_participants
0,39428300,F,40,Latino,Grad,<25K,North,City Hotel,2015-07-01,2015-07-02,...,Yes,10,218,1,7,2015,2,1,41,4
1,77491756,F,49,Latino,Mid-School,50K -- 100K,East,City Hotel,2015-07-01,2015-07-02,...,No,0,185,1,7,2015,2,1,36,6
2,73747291,F,42,caucasian,Grad,<25K,East,City Hotel,2015-07-02,2015-07-06,...,No,0,119,2,7,2015,3,4,3,6
3,67301739,M,25,African American,College,>100K,South,Airport Hotels,2015-07-02,2015-07-03,...,Yes,5,144,2,7,2015,3,1,12,7
4,77222321,F,62,Latino,High-School,25K --50K,East,Resort,2015-07-03,2015-07-04,...,Yes,10,242,3,7,2015,4,1,13,2


In [29]:
nw3_df.columns

Index(['Reservation-id', 'Gender', 'Age', 'Ethnicity', 'Educational_Level',
       'Income', 'Country_region', 'Hotel_Type', 'Expected_checkin',
       'Expected_checkout', 'Booking_date', 'Adults', 'Children', 'Babies',
       'Meal_Type', 'Visted_Previously', 'Previous_Cancellations',
       'Deposit_type', 'Booking_channel', 'Required_Car_Parking',
       'Reservation_Status', 'Use_Promotion', 'Discount_Rate', 'Room_Rate',
       'day_Expected_checkin', 'month_Expected_checkin',
       'year_Expected_checkin', 'dayofweek_Expected_checkin', 'stay',
       'booking_to_checkin', 'total_participants'],
      dtype='object')

In [30]:
col_mask = ['Gender', 'Age', 'Ethnicity', 'Educational_Level',
       'Income', 'Country_region', 'Hotel_Type',
       'Meal_Type', 'Visted_Previously', 'Previous_Cancellations',
       'Deposit_type', 'Booking_channel', 'Required_Car_Parking',
       'Reservation_Status', 'Use_Promotion', 'Room_Rate',
       'month_Expected_checkin', 'dayofweek_Expected_checkin', 'stay',
       'booking_to_checkin', 'total_participants']

In [31]:

nw4_df = nw3_df.loc[:, col_mask]

In [32]:
nw4_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27499 entries, 0 to 27498
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Gender                      27499 non-null  object
 1   Age                         27499 non-null  int64 
 2   Ethnicity                   27499 non-null  object
 3   Educational_Level           27499 non-null  object
 4   Income                      27499 non-null  object
 5   Country_region              27499 non-null  object
 6   Hotel_Type                  27499 non-null  object
 7   Meal_Type                   27499 non-null  object
 8   Visted_Previously           27499 non-null  object
 9   Previous_Cancellations      27499 non-null  object
 10  Deposit_type                27499 non-null  object
 11  Booking_channel             27499 non-null  object
 12  Required_Car_Parking        27499 non-null  object
 13  Reservation_Status          27499 non-null  ob

In [33]:
cat_cols = ['Gender', 'Ethnicity', 'Educational_Level',
       'Income', 'Country_region', 'Hotel_Type',
       'Meal_Type', 'Visted_Previously', 'Previous_Cancellations',
       'Deposit_type', 'Booking_channel', 'Required_Car_Parking',
       'Reservation_Status', 'Use_Promotion']